In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
from google.colab import drive
import os

#Caminho base do seu projeto
base_path = '/content/drive/MyDrive/projetoia'

#Listar os arquivos da pasta
print(f"\nArquivos encontrados em {base_path}:\n")
files = os.listdir(base_path)
for f in files:
    print(f)

#Verificar se os arquivos de treino e teste estão presentes
train_path = os.path.join(base_path, 'train.csv')
test_path = os.path.join(base_path, 'test.csv')

print("\nVerificando existência dos arquivos:")
print(f"Train existe? {'YES' if os.path.exists(train_path) else 'NOT'} ({train_path})")
print(f"Test existe?  {'YES' if os.path.exists(test_path) else 'NOT'} ({test_path})")



Arquivos encontrados em /content/drive/MyDrive/projetoia:

sample_submission.csv
test.csv
train.csv
sample_submission_binario.csv
submission_random_forest.csv

Verificando existência dos arquivos:
Train existe? YES (/content/drive/MyDrive/projetoia/train.csv)
Test existe?  YES (/content/drive/MyDrive/projetoia/test.csv)


In [4]:
import pandas as pd

# Caminho do arquivo
path = '/content/drive/MyDrive/projetoia/sample_submission.csv'

# Lê o CSV
df = pd.read_csv(path)

print("Antes da transformação:")
print(df.head())

# Substitui as classes por valores binários
df['target'] = df['target'].map({'attack': 1, 'normal': 0})

print("\nDepois da transformação:")
print(df.head())

#Salva o novo arquivo transformado
output_path = '/content/drive/MyDrive/projetoia/sample_submission_binario.csv'
df.to_csv(output_path, index=False)
print(f"\nArquivo salvo em: {output_path}")

Antes da transformação:
      id  target
0  24998  attack
1  12486  attack
2   5285  attack
3   5534  attack
4  29985  attack

Depois da transformação:
      id  target
0  24998       1
1  12486       1
2   5285       1
3   5534       1
4  29985       1

Arquivo salvo em: /content/drive/MyDrive/projetoia/sample_submission_binario.csv


In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

#Carregar os datasets
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))

print("Shape do DataFrame de treino:", train_df.shape)
print("Colunas do DataFrame de treino:", train_df.columns.tolist())
print("Shape do DataFrame de teste:", test_df.shape)
print("Colunas do DataFrame de teste:", test_df.columns.tolist())

#Identificar colunas de features comuns entre treino e teste
common_features = list(set(train_df.columns) & set(test_df.columns))
if 'id' in common_features:
    common_features.remove('id')

# Separar features (X) e target (y) do dataset de treino original
X = train_df[common_features]
y = train_df['target']

# Dividir o dataset de treino em conjuntos de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_test_submission = test_df[common_features]

# === Pré-processamento simples: tratar valores ausentes (se houver) ===
# Usar a média para preencher NaNs. Em um cenário real, uma estratégia mais sofisticada pode ser necessária.
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_val.mean())
X_test_submission = X_test_submission.fillna(X_test_submission.mean())

# Inicializar e treinar o modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=150, random_state=72, n_jobs=-1) # n_jobs=-1 usa todos os cores
rf_model.fit(X_train, y_train)

# Fazer previsões no dataset de validação para calcular a acurácia
val_predictions = rf_model.predict(X_val)

# Calcular e exibir a acurácia
accuracy = accuracy_score(y_val, val_predictions)
print(f"\nAcurácia do modelo no conjunto de validação: {accuracy:.4f}")

# Fazer previsões no dataset de teste para submissão
predictions_for_submission = rf_model.predict(X_test_submission)

# Criar um DataFrame para o arquivo de submissão
submission_df = pd.DataFrame({'id': test_df['id'], 'target': predictions_for_submission})

# Salvar o arquivo de submissão
output_submission_path = os.path.join(base_path, 'submission_random_forest.csv')
submission_df.to_csv(output_submission_path, index=False)

print(f"\nModelo Random Forest treinado e previsões salvas em: {output_submission_path}")
print("Primeiras 5 linhas do arquivo de submissão:")
print(submission_df.head())

Shape do DataFrame de treino: (28992, 526)
Colunas do DataFrame de treino: ['id', 'f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f12